<a href="https://colab.research.google.com/github/puneetg061/MultiLable-Images-Classification/blob/main/Document_QA_with_GPT_4o_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers langchain langchain-community langchain-openai faiss-cpu unstructured unstructured[pdf]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-m

In [1]:
import os

from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [2]:
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

In [3]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
# load the document
loader = UnstructuredFileLoader("/content/attention_is_all_you_need.pdf")
documents = loader.load()

In [ ]:
type(documents[0])

langchain_core.documents.base.Document

In [ ]:
# create text chunks

text_splitter = CharacterTextSplitter(separator='/n',
                                      chunk_size=1000,
                                      chunk_overlap=200)

text_chunks = text_splitter.split_documents(documents)

In [ ]:
# loading the vector embedding model
embeddings = HuggingFaceEmbeddings()
# vector embedding for text chunks
knowledge_base = FAISS.from_documents(text_chunks, embeddings)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# chain for aq retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever())

In [ ]:
question = "What is this document about?"
response = qa_chain.invoke({"query": question})
print(response["result"])

This document is a research paper titled "Attention Is All You Need," authored by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin. It introduces the Transformer model, a novel neural network architecture for sequence transduction tasks such as machine translation. The Transformer model relies entirely on attention mechanisms, dispensing with recurrence and convolutions entirely. The paper demonstrates that the Transformer achieves superior performance in terms of translation quality and training efficiency compared to previous models based on recurrent or convolutional neural networks. The authors also discuss the model's architecture, training process, and results on various tasks, including machine translation and English constituency parsing.


In [ ]:
response

{'query': 'What is this document about?',
 'result': 'This document is a research paper titled "Attention Is All You Need," authored by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin. It introduces the Transformer model, a novel neural network architecture for sequence transduction tasks such as machine translation. The Transformer model relies entirely on attention mechanisms, dispensing with recurrence and convolutions entirely. The paper demonstrates that the Transformer achieves superior performance in terms of translation quality and training efficiency compared to previous models based on recurrent or convolutional neural networks. The authors also discuss the model\'s architecture, training process, and results on various tasks, including machine translation and English constituency parsing.'}

In [ ]:
question = "What is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": question})
response["result"]

"The model architecture discussed in the paper is the Transformer. The Transformer is a novel network architecture that relies entirely on attention mechanisms, dispensing with recurrence and convolutions entirely. It follows an encoder-decoder structure, where both the encoder and decoder are composed of stacks of identical layers. Each layer in the encoder consists of a multi-head self-attention mechanism followed by a position-wise fully connected feed-forward network, with residual connections and layer normalization applied around each sub-layer. The decoder has an additional sub-layer that performs multi-head attention over the encoder's output. The architecture allows for significantly more parallelization and achieves state-of-the-art results in machine translation tasks."

In [ ]:
question = "What are the applications of attention in our Model?"
response = qa_chain.invoke({"query": question})
response["result"]

'The Transformer model uses multi-head attention in three different ways:\n\n1. **Encoder-Decoder Attention**: In these layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mechanism mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models.\n\n2. **Self-Attention in the Encoder**: In these layers, all of the keys, values, and queries come from the same place, specifically the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder.\n\n3. **Self-Attention in the Decoder**: Similar to the encoder, these layers allow each position in the decoder to attend to all positions in the decoder up to and including that position. To preserve the auto-regressive property, the model prevents positions from attending to 

In [ ]:
print(response["result"])

The Transformer model uses multi-head attention in three different ways:

1. **Encoder-Decoder Attention**: In these layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mechanism mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models.

2. **Self-Attention in the Encoder**: In these layers, all of the keys, values, and queries come from the same place, specifically the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder.

3. **Self-Attention in the Decoder**: Similar to the encoder, these layers allow each position in the decoder to attend to all positions in the decoder up to and including that position. To preserve the auto-regressive property, the model prevents positions from attending to subsequ